# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from scipy.stats import linregress
import time


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df = pd.read_csv("city_data.csv")
del city_data_df["Unnamed: 0"]
city_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Katangli,0,RU,1600017124,91,51.71,143.23,48.22,6.69
1,Punta Arenas,40,CL,1600016984,61,-53.15,-70.92,46.40,16.11
2,Conceição do Araguaia,69,BR,1600017124,12,-8.26,-49.26,106.86,4.07
3,Albany,58,US,1600017096,64,42.60,-73.97,73.99,1.01
4,Bluff,100,NZ,1600017036,98,-46.60,168.33,46.99,5.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = city_data_df[["Lat","Lng"]].astype(float)
humidity = city_data_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [26]:
under_MT_city_data_df = city_data_df.loc[city_data_df["Max Temp"] < 80, :]
over_MT_city_data_df = under_MT_city_data_df.loc[under_MT_city_data_df["Max Temp"] > 70, :]
WS_city_data_df = over_MT_city_data_df.loc[over_MT_city_data_df["Wind Speed"] < 10, :]
cloud_city_data_df = WS_city_data_df.loc[WS_city_data_df["Cloudiness"] == 0, :]
humidity_city_data_df = cloud_city_data_df.loc[cloud_city_data_df["Humidity"] <60]
perfect_vacation = humidity_city_data_df.reset_index()
perfect_vacation.head()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,140,Atherton,0,GB,1600017152,51,53.52,-2.49,75.99,3.00
1,237,San Quintín,0,MX,1600016770,52,30.48,-115.95,78.06,7.78
2,246,Rundu,0,NaN,1600017176,18,-17.93,19.77,73.92,2.30
3,302,Otjiwarongo,0,NaN,1600017188,20,-20.46,16.65,70.20,5.61
4,476,Saýat,0,TM,1600017227,35,38.78,63.88,71.60,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = perfect_vacation
hotel_df['Hotel Name'] = np.nan

In [37]:
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json["results"]
    if len(results) > 0:
        name = response_json["results"][0]["name"]
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

narrowed_city_df = hotel_df
narrowed_city_df.head()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,140,Atherton,0,GB,1600017152,51,53.52,-2.49,75.99,3.00,Hollins
1,237,San Quintín,0,MX,1600016770,52,30.48,-115.95,78.06,7.78,Old Mill Hotel
2,246,Rundu,0,NaN,1600017176,18,-17.93,19.77,73.92,2.30,Hakusembe River Lodge
3,302,Otjiwarongo,0,NaN,1600017188,20,-20.46,16.65,70.20,5.61,Cest Si Bon Hotel
4,476,Saýat,0,TM,1600017227,35,38.78,63.88,71.60,6.93,NaN


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))